In [2]:
import numpy as np
import pandas as pd
 
import keras
import keras.backend as K
from keras.layers import Input, Convolution2D, Activation, MaxPooling2D, Dense, BatchNormalization, Dropout
from keras.layers.core import Flatten
from keras.optimizers import SGD
from keras.models import Model
from keras.utils import np_utils
from keras.constraints import maxnorm
from keras.regularizers import l2
from keras.callbacks import LearningRateScheduler
from keras.layers.normalization import BatchNormalization

print(keras.__version__)

Using TensorFlow backend.


2.1.4


In [3]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

In [15]:
N_train = 30000
training_inputs = X_train[0:N_train,:,:] / 255.0
training_targets = np_utils.to_categorical(y_train)[0:N_train]

val_inputs = X_train[(N_train+1):42000,:,:] / 255.0
val_targets = np_utils.to_categorical(y_train)[(N_train+1):42000]

In [18]:
training_inputs = training_inputs.reshape(training_inputs.shape[0], 784)
val_inputs = val_inputs.reshape(val_inputs.shape[0], 784)

### Layer by layer pretraining Models (greedy layer-wise training)

#### Layer 1

In [20]:
input_img = Input(shape = (784, ))
distorted_input1 = Dropout(.1)(input_img)
encoded1 = Dense(800, activation = 'sigmoid')(distorted_input1)
encoded1_bn = BatchNormalization()(encoded1)
decoded1 = Dense(784, activation = 'sigmoid')(encoded1_bn)

autoencoder1 = Model(input=input_img, output=decoded1)
encoder1 = Model(input=input_img, output=encoded1_bn)

C:\Users\jhpark\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
C:\Users\jhpark\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ba...)`


#### Layer 2

In [21]:
encoded1_input = Input(shape = (800,))
distorted_input2 = Dropout(.2)(encoded1_input)
encoded2 = Dense(400, activation='sigmoid')(distorted_input2)
encoded2_bn = BatchNormalization()(encoded2)
decoded2 = Dense(800, activation='sigmoid')(encoded2_bn)

autoencoder2 = Model(input=encoded1_input, output=decoded2)
encoder2 = Model(input=encoded1_input, output=encoded2_bn)

C:\Users\jhpark\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
C:\Users\jhpark\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ba...)`


#### Layer 3

In [23]:
encoded2_input = Input(shape = (400,))
distorted_input3 = Dropout(.3)(encoded2_input)
encoded3 = Dense(200, activation='sigmoid')(distorted_input3)
encoded3_bn = BatchNormalization()(encoded3)
decoded3 = Dense(400, activation='sigmoid')(encoded3_bn)

autoencoder3 = Model(input=encoded2_input, output=decoded3)
encoder3 = Model(input=encoded2_input, output=encoded3_bn)

C:\Users\jhpark\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
C:\Users\jhpark\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ba...)`


#### Deep Autoencoder

In [24]:
encoded1_da = Dense(800, activation='sigmoid')(input_img)
encoded1_da_bn = BatchNormalization()(encoded1_da)
encoded2_da = Dense(400, activation='sigmoid')(encoded1_da_bn)
encoded2_da_bn = BatchNormalization()(encoded2_da)
encoded3_da = Dense(200, activation='sigmoid')(encoded2_da_bn)
encoded3_da_bn = BatchNormalization()(encoded3_da)
decoded3_da = Dense(400, activation='sigmoid')(encoded3_da_bn)
decoded2_da = Dense(800, activation='sigmoid')(decoded3_da)
decoded1_da = Dense(784, activation='sigmoid')(decoded2_da)

deep_autoencoder = Model(input=input_img, output=decoded1_da)

C:\Users\jhpark\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [25]:
nad_encoded1_da = Dense(800, activation='sigmoid')(input_img)
nad_encoded1_da_bn = BatchNormalization()(nad_encoded1_da)
nad_encoded2_da = Dense(400, activation='sigmoid')(nad_encoded1_da_bn)
nad_encoded2_da_bn = BatchNormalization()(nad_encoded2_da)
nad_decoded2_da = Dense(800, activation='sigmoid')(nad_encoded2_da_bn)
nad_decoded1_da = Dense(784, activation='sigmoid')(nad_decoded2_da)

nad_deep_autoencoder = Model(input=input_img, output=nad_decoded1_da)

C:\Users\jhpark\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [26]:
sgd1 = SGD(lr = 5, decay = 0.5, momentum = 0.85, nesterov = True)
sgd2 = SGD(lr = 5, decay = 0.5, momentum = 0.85, nesterov = True)
sgd3 = SGD(lr = 5, decay = 0.5, momentum = 0.85, nesterov = True)

autoencoder1.compile(loss='binary_crossentropy', optimizer = sgd1)
autoencoder2.compile(loss='binary_crossentropy', optimizer = sgd2)
autoencoder3.compile(loss='binary_crossentropy', optimizer = sgd3)

deep_autoencoder.compile(loss='binary_crossentropy', optimizer = sgd1)
nad_deep_autoencoder.compile(loss='binary_crossentropy', optimizer=sgd1)

# what will happen to the learning rates under this decay schedule?
lr = 5
for i in range(12):
    lr = lr - lr*.15
    print(lr)

4.25
3.6125
3.0706249999999997
2.6100312499999996
2.2185265624999997
1.8857475781249997
1.60288544140625
1.3624526251953124
1.1580847314160156
0.9843720217036133
0.8367162184480713
0.7112087856808607


### Training first autoencoder

In [27]:
autoencoder1.fit(training_inputs, training_inputs, nb_epoch=8, batch_size=512, validation_split=0.3, shuffle=True)

C:\Users\jhpark\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':


Train on 21000 samples, validate on 9000 samples
Epoch 1/8
21000/21000 [==============================] - 4s 169us/step - loss: 0.5178 - val_loss: 0.3335
Epoch 2/8
21000/21000 [==============================] - 3s 148us/step - loss: 0.2945 - val_loss: 0.2196
Epoch 3/8
21000/21000 [==============================] - 3s 147us/step - loss: 0.2223 - val_loss: 0.1890
Epoch 4/8
21000/21000 [==============================] - 3s 149us/step - loss: 0.1944 - val_loss: 0.1756
Epoch 5/8
21000/21000 [==============================] - 3s 147us/step - loss: 0.1802 - val_loss: 0.1673
Epoch 6/8
21000/21000 [==============================] - 3s 149us/step - loss: 0.1714 - val_loss: 0.1616
Epoch 7/8
21000/21000 [==============================] - 3s 148us/step - loss: 0.1655 - val_loss: 0.1572
Epoch 8/8
21000/21000 [==============================] - 3s 148us/step - loss: 0.1611 - val_loss: 0.1540


In [28]:
first_layer_code = encoder1.predict(training_inputs)
print(first_layer_code.shape)

(30000, 800)


### Training second autoencoder
encoder1에서 나온 first_layer_code를 second autoencoder의 input 및 output으로 넣어서 학습시킴

In [29]:
autoencoder2.fit(first_layer_code, first_layer_code, nb_epoch=8, batch_size=512, validation_split=0.25, shuffle=True)

C:\Users\jhpark\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':


Train on 22500 samples, validate on 7500 samples
Epoch 1/8
22500/22500 [==============================] - 2s 101us/step - loss: -1.4606 - val_loss: -3.1380
Epoch 2/8
22500/22500 [==============================] - 2s 93us/step - loss: -3.9531 - val_loss: -4.5780
Epoch 3/8
22500/22500 [==============================] - 2s 91us/step - loss: -4.7212 - val_loss: -4.8903
Epoch 4/8
22500/22500 [==============================] - 2s 90us/step - loss: -4.9186 - val_loss: -5.0081
Epoch 5/8
22500/22500 [==============================] - 2s 90us/step - loss: -5.0123 - val_loss: -5.0789
Epoch 6/8
22500/22500 [==============================] - 2s 91us/step - loss: -5.0713 - val_loss: -5.1299
Epoch 7/8
22500/22500 [==============================] - 2s 91us/step - loss: -5.1163 - val_loss: -5.1693
Epoch 8/8
22500/22500 [==============================] - 2s 93us/step - loss: -5.1509 - val_loss: -5.2015


In [30]:
second_layer_code = encoder2.predict(first_layer_code)
print(second_layer_code.shape)

(30000, 400)


### Training third autoencoder
encoder2에서 나온 second_layer_code를 third autoencoder의 input 및 output으로 넣어서 학습시킴

In [31]:
autoencoder3.fit(second_layer_code, second_layer_code, nb_epoch=8, batch_size=512, validation_split=0.3, shuffle=True)

C:\Users\jhpark\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':


Train on 21000 samples, validate on 9000 samples
Epoch 1/8
21000/21000 [==============================] - 1s 45us/step - loss: -11.3211 - val_loss: -15.7456
Epoch 2/8
21000/21000 [==============================] - 1s 38us/step - loss: -15.7859 - val_loss: -16.1216
Epoch 3/8
21000/21000 [==============================] - 1s 39us/step - loss: -16.0022 - val_loss: -16.2525
Epoch 4/8
21000/21000 [==============================] - 1s 37us/step - loss: -16.1033 - val_loss: -16.3328
Epoch 5/8
21000/21000 [==============================] - 1s 38us/step - loss: -16.1723 - val_loss: -16.3914
Epoch 6/8
21000/21000 [==============================] - 1s 35us/step - loss: -16.2243 - val_loss: -16.4373
Epoch 7/8
21000/21000 [==============================] - 1s 36us/step - loss: -16.2621 - val_loss: -16.4742
Epoch 8/8
21000/21000 [==============================] - 1s 34us/step - loss: -16.2951 - val_loss: -16.5043


### Setting the weights of the deep autoencoder

In [41]:
deep_autoencoder.layers[1].set_weights(autoencoder1.layers[2].get_weights()) # first dense layer (800)
deep_autoencoder.layers[2].set_weights(autoencoder1.layers[3].get_weights()) # first bn layer
deep_autoencoder.layers[3].set_weights(autoencoder2.layers[2].get_weights()) # second dense layer
deep_autoencoder.layers[4].set_weights(autoencoder2.layers[3].get_weights()) # second bn layer
deep_autoencoder.layers[5].set_weights(autoencoder3.layers[2].get_weights()) # third dense layer
deep_autoencoder.layers[6].set_weights(autoencoder3.layers[3].get_weights()) # third bn layer
deep_autoencoder.layers[7].set_weights(autoencoder3.layers[4].get_weights()) # first decoder
deep_autoencoder.layers[8].set_weights(autoencoder2.layers[4].get_weights()) # second decoder
deep_autoencoder.layers[9].set_weights(autoencoder1.layers[4].get_weights()) # third decoder

### Setting up the weights of the not-as-deep autoencoder

In [42]:
nad_deep_autoencoder.layers[1].set_weights(autoencoder1.layers[2].get_weights()) # first dense layer
nad_deep_autoencoder.layers[2].set_weights(autoencoder1.layers[3].get_weights()) # first bn layer
nad_deep_autoencoder.layers[3].set_weights(autoencoder2.layers[2].get_weights()) # second dense layer
nad_deep_autoencoder.layers[4].set_weights(autoencoder2.layers[3].get_weights()) # second bn layer
nad_deep_autoencoder.layers[5].set_weights(autoencoder2.layers[4].get_weights()) # second decoder
nad_deep_autoencoder.layers[6].set_weights(autoencoder1.layers[4].get_weights()) # third decoder

In [43]:
decoded_inputs = nad_deep_autoencoder.predict(training_inputs[0:25,])

In [45]:
decoded_inputs

array([[ 0.15118212,  0.18637228,  0.1645885 , ...,  0.13445844,
         0.22603853,  0.29167011],
       [ 0.21193783,  0.2805427 ,  0.17357469, ...,  0.18749757,
         0.22023115,  0.38302997],
       [ 0.16627672,  0.21487252,  0.253699  , ...,  0.13204175,
         0.18625297,  0.16529225],
       ..., 
       [ 0.20453706,  0.15460625,  0.20954278, ...,  0.13629699,
         0.13651608,  0.22064735],
       [ 0.27779889,  0.19214535,  0.2262421 , ...,  0.1863195 ,
         0.19440638,  0.4094469 ],
       [ 0.22172701,  0.25243217,  0.15116775, ...,  0.11172511,
         0.19175291,  0.22743715]], dtype=float32)

## On to 'fine-tuning' for classification 
전체 모델을 supervised learning으로 fine-tuning

In [48]:
dense1 = Dense(500, activation='relu')(nad_decoded1_da)
dense1_drop = Dropout(.3)(dense1)
dense2 = Dense(10, activation='sigmoid')(dense1_drop)

classifier = Model(input=input_img, output=dense2)
sgd4 = SGD(lr=.1, decay=0.001, momentum=0.95, nesterov=True)
classifier.compile(loss='categorical_crossentropy', optimizer=sgd4, metrics=['accuracy'])

classifier.fit(training_inputs, training_targets, nb_epoch=6, batch_size=600, validation_split=0.25, shuffle=True)

C:\Users\jhpark\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
C:\Users\jhpark\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 22500 samples, validate on 7500 samples
Epoch 1/6
22500/22500 [==============================] - 5s 217us/step - loss: 1.1087 - acc: 0.6598 - val_loss: 0.4034 - val_acc: 0.8712
Epoch 2/6
22500/22500 [==============================] - 4s 188us/step - loss: 0.2995 - acc: 0.9072 - val_loss: 0.2508 - val_acc: 0.9213
Epoch 3/6
22500/22500 [==============================] - 4s 195us/step - loss: 0.1922 - acc: 0.9412 - val_loss: 0.1989 - val_acc: 0.9372
Epoch 4/6
22500/22500 [==============================] - 4s 196us/step - loss: 0.1378 - acc: 0.9576 - val_loss: 0.1837 - val_acc: 0.9433
Epoch 5/6
22500/22500 [==============================] - 4s 191us/step - loss: 0.1097 - acc: 0.9668 - val_loss: 0.1557 - val_acc: 0.9531
Epoch 6/6
22500/22500 [==============================] - 4s 188us/step - loss: 0.0819 - acc: 0.9752 - val_loss: 0.1577 - val_acc: 0.9501


In [49]:
val_preds = classifier.predict(val_inputs)

In [51]:
predictions = np.argmax(val_preds, axis=1)
true_digits = np.argmax(val_targets, axis=1)
predictions[0:25]

array([7, 3, 9, 6, 4, 4, 1, 7, 0, 9, 3, 5, 8, 2, 7, 9, 7, 7, 8, 5, 7, 9, 6,
       6, 6], dtype=int64)

In [52]:
true_digits[0:25]

array([7, 3, 9, 6, 4, 4, 1, 7, 0, 9, 3, 5, 8, 2, 7, 4, 7, 7, 8, 5, 7, 9, 6,
       6, 6], dtype=int64)